In [ ]:
import pandas as pd
import scanpy as sc
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
mpl.rcParams["figure.figsize"] = (16, 12)

# Fixing metadata part

In [ ]:
ds = sc.read_h5ad("../data/02dataset/integrated.h5ad")

In [ ]:
ds.obs

In [ ]:
ds.obs["Cell Population"] = ds.obs["Cell Population"].astype(str)
ds.obs["Cell Population"] = ds.obs["Cell Population"].replace("Stromal_Myeloid", "Stromal, Myeloid")
ds.obs["Cell Population"] = ds.obs["Cell Population"].astype("category")

In [ ]:
ds.obs["Cell Population"].value_counts(dropna=False)

In [ ]:
sample_to_name = {
    "Sample_1": "Donor 1",
    "Sample_2": "Donor 1",
    "Sample_3": "Donor 1",
    "Sample_4": "Donor 1",
    "Sample_5": "Donor 1",
    "Sample_6": "Donor 1",
    "Sample_7": "PMB 1",
    "Sample_8": "PMB 1",
    "Sample_9": "PMB 1",
    "Sample_10": "PMB 1",
    "Sample_11": "Donor 2",
    "Sample_12": "Donor 2",
    "Sample_13": "Donor 2",
    "Sample_14": "Donor 2",
    "Sample_15": "Case 1",
    "Sample_16": "Case 1",
    "Sample_17": "Case 1",
    "Sample_18": "Case 1",
    "Sample_19": "PMB 2",
    "Sample_20": "PMB 2",
    "Sample_21": "PMB 2",
    "Sample_22": "PMB 2",
}

In [ ]:
ds.obs["Sample Name"] = ds.obs["orig.ident"].replace(sample_to_name)

In [ ]:
ds.obs["Sample Name"].value_counts()

In [ ]:
ds.write_h5ad("../data/02dataset/integrated-manual.h5ad")

# Plots

In [ ]:
sc.pl.umap(ds, color="leiden", size=10, legend_loc="on data", title="By Cluster")

In [ ]:
sc.pl.umap(ds, color="leiden", size=10, legend_loc="on data", title="By Cluster")

In [ ]:
markers = [
    "CD3E", "FOXJ1", "FABP4", "PTPRC", "CDH1", "PDGFRB", "ACTA2", "CLDN5",
    "AGER", "MRC1", "SFTPC", "PROX1"
]
sc.pl.stacked_violin(
    ds,
    markers,
    groupby="leiden",
    rotation=90,
    figsize=(10, 12)
);

## Broad markers

In [ ]:
sc.pl.umap(ds, color="FABP4", size=10)

In [ ]:
sc.pl.umap(ds, color="MRC1", size=10)

In [ ]:
sc.pl.umap(ds, color="EPCAM", size=10)

In [ ]:
sc.pl.umap(ds, color="PDGFRA", size=10)

## Technical variables

In [ ]:
sc.pl.umap(ds, color="total_counts", size=10)

In [ ]:
sc.pl.umap(ds, color="pct_counts_mito", size=10)

In [ ]:
sc.pl.umap(ds, color="pct_counts_ribo", size=10)

In [ ]:
sc.pl.umap(ds, color="MKI67", size=10)

## Interesting genes

In [ ]:
sc.pl.umap(ds, color="IL1B", size=10)

In [ ]:
sc.pl.umap(ds, color="IL6", size=10)

In [ ]:
sc.pl.umap(ds, color="CCL20", size=10)

## Metadata

In [ ]:
sc.pl.umap(ds, color="Cell Population", size=10)

In [ ]:
sc.pl.umap(ds, color="Sample Name", size=10)

In [ ]:
sc.pl.umap(ds, color="COVID-19", size=10)

In [ ]:
sc.pl.umap(ds, color="Tissue Type", size=10)

## Composition

In [ ]:
def plot_composition(ds, group_by, color):
    bottom = np.zeros(len(ds.obs[group_by].unique()))
    fig, ax = plt.subplots()
    for s in ds.obs[color].unique():
        cnt = ds.obs[group_by][ds.obs[color] == s].value_counts().sort_index()
        ax.bar(cnt.index, cnt, bottom=bottom, label=s)
        bottom += cnt
    ax.legend()
    fig.suptitle(f"{group_by} by {color}")

In [ ]:
plot_composition(ds, "leiden", color="orig.ident")

In [ ]:
plot_composition(ds, "leiden", color="Cell Population")

In [ ]:
plot_composition(ds, "leiden", color="Sample Name")

In [ ]:
plot_composition(ds, "leiden", color="COVID-19")

# Differential gene expression COVID vs non-COVID

In [ ]:
def get_markers(anndata, groupby):
    def calc_pct_1(x):
        cells = anndata.obs[groupby] == x.cluster
        gene = anndata.var_names == x.gene
        return (anndata.X[cells, gene] > 0).sum() / cells.sum()

    def calc_pct_2(x):
        cells = anndata.obs[groupby] != x.cluster
        gene = anndata.var_names == x.gene
        return (anndata.X[cells, gene] > 0).sum() / cells.sum()
    
    markers = pd.concat([
        pd.DataFrame(anndata.uns["rank_genes_groups"]["names"]).melt(),
        pd.DataFrame(anndata.uns["rank_genes_groups"]["pvals_adj"]).melt(),
        pd.DataFrame(anndata.uns["rank_genes_groups"]["logfoldchanges"]).melt()
    ], axis=1)
    markers.columns = ("cluster", "gene", "cluster2", "pval_adj", "cluster3", "logFC")
    markers = markers.loc[:, ["cluster", "gene", "logFC", "pval_adj"]]
    #markers = markers.loc[markers.logFC > 0, ]
    markers = markers.loc[markers.pval_adj < 0.05, ]
    markers["pct.1"] = markers.apply(calc_pct_1, axis=1)
    markers["pct.2"] = markers.apply(calc_pct_2, axis=1)
    markers = markers.loc[:, ["cluster", "gene", "logFC", "pct.1", "pct.2", "pval_adj"]]
    return markers

In [ ]:
ds.obs["COVID-19"] = ds.obs["COVID-19"].astype(str).map({"True": "COVID+", "False": "COVID-"}).astype("category").values

In [ ]:
ds_slice = ds[ds.obs["leiden"] == "0", :]
sc.tl.rank_genes_groups(ds_slice, groupby="COVID-19", method="wilcoxon")

In [ ]:
markers = get_markers(ds_slice, "COVID-19")
markers.loc[markers.cluster == "COVID-", "logFC"] *= -1
markers.sort_values("logFC")

In [ ]:
result = []
for cl in ds.obs.leiden.unique():
    ds_slice = ds[ds.obs["leiden"] == cl, :]
    cnt = ds_slice.obs["COVID-19"].value_counts()[0] / ds_slice.shape[0]
    if cnt < 0.05 or cnt > 0.95:
        continue
    sc.tl.rank_genes_groups(ds_slice, groupby="COVID-19", method="wilcoxon")
    markers = get_markers(ds_slice, "COVID-19")
    markers.loc[markers.cluster == "COVID-", "logFC"] *= -1
    markers["COVID"] = markers.cluster
    markers.cluster = cl
    markers = markers.loc[:, ["cluster", "COVID", "gene", "logFC", "pct.1", "pct.2", "pval_adj"]]
    result.append(markers.sort_values("logFC"))

In [ ]:
all_degs = pd.concat(result, ignore_index=True)
all_degs.cluster = all_degs.cluster.astype(int)
all_degs.sort_values(["cluster", "logFC"]).to_csv("01covid-noncovid-markers.csv")